# PyTorch - Night to Day
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/PyTorch/PyTorch_Night_to_Day.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

### Input

### Install torchvision

In [1]:
pip install torchvision

  Using cached torchvision-0.10.0-cp38-cp38-manylinux1_x86_64.whl (22.1 MB)
     |████████████████████████████████| 831.4 MB 4.8 MB/s eta 0:00:0145  |▏                               | 4.1 MB 24.1 MB/s eta 0:00:3533     |████▍                           | 114.1 MB 6.1 MB/s eta 0:01:59     |████▍                           | 115.0 MB 6.1 MB/s eta 0:01:5850197.3 MB 12.6 MB/s eta 0:00:51MB 99.0 MB/s eta 0:00:07�▋                      | 248.8 MB 99.0 MB/s eta 0:00:061.4 MB 99.0 MB/s eta 0:00:06��███▉                      | 256.1 MB 99.0 MB/s eta 0:00:06| 258.0 MB 99.0 MB/s eta 0:00:06�█████                      | 260.9 MB 99.0 MB/s eta 0:00:06     |██████████▊                     | 278.8 MB 32.8 MB/s eta 0:00:17/s eta 0:00:17     |███████████▏                    | 291.3 MB 32.8 MB/s eta 0:00:17     |███████████▍                    | 297.2 MB 32.8 MB/s eta 0:00:17     |████████████                    | 312.4 MB 9.2 MB/s eta 0:00:57     |████████████▋                   | 327.5 MB 9.2 MB/s eta 0

### Import library

In [2]:
import torch
from torch import nn
import PIL
import requests
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms as T
from model import Generator_Resnet

ModuleNotFoundError: No module named 'torchvision'

### Variables

In [ ]:
img_url = "https://lh3.googleusercontent.com/Mjz79x3QRUldbe6VJhfw8xxWcvnZ9YT0P3KJxT1KdUzolrovEMxuUjoti8Xtk3uDq5ntWr_04C2KtU6wqg3b=w2880-h1642"

### Image of Dimension 300 x 300 ?

In [3]:
in_img = Image.open(requests.get(img_url, stream=True).raw) # change the path to the input image

transform = T.Compose([
    T.ToTensor(),
    T.Resize((300, 300))
    ])

in_img = torch.reshape(transform(in_img), (1, 3, 300, 300))

### Explaination ?

In [4]:
_, ax = plt.subplots(figsize=(5, 5))
ax.axis("off")
ax.imshow(np.array(in_img).reshape(3, 300, 300).transpose(1, 2, 0), aspect="auto", interpolation="nearest")

### Model

### Cycle Generative Adversarial Networks (GAN) with MSE and L1 loss

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        # print(in_features)

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.LeakyReLU(inplace=True),
            nn.ReflectionPad2d(2),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.Conv2d(in_features, in_features, 3)
        )

    def forward(self, x):
        out = self.block(x)
        return x + out


class Generator_Resnet(nn.Module):
    def __init__(self, in_shape, n_res_blocks=5):
        super(Generator_Resnet, self).__init__()
        channels = in_shape[0]
        n_blocks = n_res_blocks

        out_features = 64

        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.LeakyReLU(inplace=True)
        ]
        in_features = out_features

        # Downscaling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.LeakyReLU(inplace=True)
            ]
            in_features = out_features

        model += [ResidualBlock(in_features) for _ in range(n_blocks)]

        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

            # Output layer
        # print(out_features)
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

### Explaination ?

In [6]:
Gen_Night2Day = Generator_Resnet((3, 300, 300), 4)
Gen_Night2Day.load_state_dict(torch.load("./G_AB.pth", map_location=torch.device("cpu")))

### Output

### Image of Dimension 300 x 300 ?

In [7]:
out_img = Gen_Night2Day(in_img).detach().numpy().reshape((3, 300, 300)).transpose(1, 2, 0)

In [8]:
_, ax = plt.subplots(1, 2, figsize=(15, 8))
ax[0].axis('off')
ax[1].axis('off')
ax[0].set_title("Input Night image", fontsize=20)
ax[1].set_title("Output Day image", fontsize=20)
ax[0].imshow(np.array(in_img).reshape(3, 300, 300).transpose(1, 2, 0), aspect="auto", interpolation="nearest")
ax[1].imshow(out_img, interpolation="nearest", aspect="auto")